# Azure Subscription

This model is based on the subscription webhooks received from the Azure Marketplace.
The landing page creates these facts.
The portal subscribes to them to determine how many replicators can be configured.

In [1]:
// Reference the Jinaga NuGet packages
#r "nuget: Jinaga, 0.12.0"
#r "nuget: Jinaga.Graphviz, 0.12.0"
#r "nuget: Jinaga.UnitTest, 0.12.0"

Installed Packages Jinaga, 0.12.0 Jinaga.Graphviz, 0.12.0 Jinaga.UnitTest, 0.12.0

In [2]:
using Jinaga;
using Jinaga.Graphviz;
using Jinaga.UnitTest;

using System.Net.Http.Headers;
using System.Threading.Tasks;

public class ExpressAuthenticationHandler : Jinaga.Http.IHttpAuthenticationProvider
{
    public readonly string cookie;

    public ExpressAuthenticationHandler(string cookie)
    {
        this.cookie = cookie;
    }

    public void SetRequestHeaders(HttpRequestHeaders headers)
    {
        headers.Add("Cookie", $"connect.sid={cookie}");
    }

    public Task<bool> Reauthenticate()
    {
        return Task.FromResult(false);
    }
}

// Create a Jinaga client for the portal
var jinagaClient = JinagaClient.Create(opt =>
{
    opt.HttpEndpoint = new Uri("https://app.jinaga.com/jinaga");
    opt.HttpAuthenticationProvider = new ExpressAuthenticationHandler("your cookie");
});

## Enterprise Application Pattern

The Azure landing page app is an enterprise application.
The top-level user is a temporary principal.
It is created to initialize the environment and the first administrator.
And then the private key is discarded.

In [3]:
[FactType("Azure.Environment")]
public record Environment(User creator, string environmentId);

Renderer.RenderTypes(typeof(Environment))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Azure.Environment -->
 
 Azure.Environment 
 
 Azure.Environment 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Azure.Environment->Jinaga.User -->
 
 Azure.Environment->Jinaga.User 
 
 
 creator

Administrators represent people.
Administrators are granted permission to add service principals and other administrators to an environment.
That permission can be revoked.

The creator of the environment grants permission to the initial administrator.
Thereafter, an administrator can add new administrators and revoke permissions.

In [4]:
[FactType("Azure.Administrator")]
public record Administrator(User user, Environment environment, DateTime createdAt);

[FactType("Azure.Administrator.Revoke")]
public record RevokeAdministrator(Administrator administrator);

Renderer.RenderTypes(typeof(Administrator), typeof(RevokeAdministrator))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Azure.Administrator -->
 
 Azure.Administrator 
 
 Azure.Administrator 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Azure.Administrator->Jinaga.User -->
 
 Azure.Administrator->Jinaga.User 
 
 
 user 
 
<!-- Azure.Environment -->
 
 Azure.Environment 
 
 Azure.Environment 
 
<!-- Azure.Administrator->Azure.Environment -->
 
 Azure.Administrator->Azure.Environment 
 
 
 environment 
 
<!-- Azure.Environment->Jinaga.User -->
 
 Azure.Environment->Jinaga.User 
 
 
 creator 
 
<!-- Azure.Administrator.Revoke -->
 
 Azure.Administrator.Revoke 
 
 Azure.Administrator.Revoke 
 
<!-- Azure.Administrator.Revoke->Azure.Administrator -->
 
 Azure.Administrator.Revoke->Azure.Administrator 
 
 
 administrator

In [5]:
// Create an environment and make the logged in user an administrator.
var (user, profile) = await jinagaClient.Login();

var environment = await jinagaClient.SingleUse(async principal =>
{
    var environment = await jinagaClient.Fact(new Environment(principal, "test"));
    await jinagaClient.Fact(new Administrator(user, environment, DateTime.UtcNow));
    return environment;
});

System.Text.Json.JsonSerializer.Serialize(environment.creator.publicKey)

"-----BEGIN PUBLIC KEY-----\r\nMIIBCgKCAQEAoCTbrASql8xsTby30NwsizfdDVrfg\u002B8B6y/4brCoyjbGMRpQgh\u002Ba\r\niDvcbNUsafgAyeUgxqOcpXmdyeHL3A4z0KjI1Ti5Us6RADSqve7yt21WZXxpGcmr\r\nYMlxwCJ5vmaxGN4DYAmWhZI7\u002B4zZ6wBj\u002BJarM8kEOz\u002BH3RMsnp05qAoLmx9an\u002Bpg\r\nQT8X7pBc24b\u002BuS9sAHl/hboQ/Rd7ivmErbvlyZiXktj8epuETM7VVn4aPMQqezT9\r\nNCQA6kHRC49pTrv9qtGPhI6JXIODG/blMPG6WtNY\u002B7zjEOYAtKf6isq53r1d0hVa\r\nHbdDnPpLQmoqY6/xi17nMRsX3GrVdt7AgQIDAQAB\r\n-----END PUBLIC KEY-----\r\n"

In [ ]:
string publicKey = "-----BEGIN PUBLIC KEY-----\r\nMIIBCgKCAQEAoCTbrASql8xsTby30NwsizfdDVrfg\u002B8B6y/4brCoyjbGMRpQgh\u002Ba\r\niDvcbNUsafgAyeUgxqOcpXmdyeHL3A4z0KjI1Ti5Us6RADSqve7yt21WZXxpGcmr\r\nYMlxwCJ5vmaxGN4DYAmWhZI7\u002B4zZ6wBj\u002BJarM8kEOz\u002BH3RMsnp05qAoLmx9an\u002Bpg\r\nQT8X7pBc24b\u002BuS9sAHl/hboQ/Rd7ivmErbvlyZiXktj8epuETM7VVn4aPMQqezT9\r\nNCQA6kHRC49pTrv9qtGPhI6JXIODG/blMPG6WtNY\u002B7zjEOYAtKf6isq53r1d0hVa\r\nHbdDnPpLQmoqY6/xi17nMRsX3GrVdt7AgQIDAQAB\r\n-----END PUBLIC KEY-----\r\n";

var environment = new Environment(
    new User(publicKey),
    "test"
);

## Service Principals

Service principals represents machines.
They can record facts about an environment.

An administrator creates service principals.
Service principals cannot grant privileges to others.

In [6]:
[FactType("Azure.ServicePrincipal")]
public record ServicePrincipal(User user, Environment environment, DateTime createdAt);

[FactType("Azure.ServicePrincipal.Revoke")]
public record RevokeServicePrincipal(ServicePrincipal servicePrincipal);

Renderer.RenderTypes(typeof(ServicePrincipal), typeof(RevokeServicePrincipal))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Azure.ServicePrincipal -->
 
 Azure.ServicePrincipal 
 
 Azure.ServicePrincipal 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Azure.ServicePrincipal->Jinaga.User -->
 
 Azure.ServicePrincipal->Jinaga.User 
 
 
 user 
 
<!-- Azure.Environment -->
 
 Azure.Environment 
 
 Azure.Environment 
 
<!-- Azure.ServicePrincipal->Azure.Environment -->
 
 Azure.ServicePrincipal->Azure.Environment 
 
 
 environment 
 
<!-- Azure.Environment->Jinaga.User -->
 
 Azure.Environment->Jinaga.User 
 
 
 creator 
 
<!-- Azure.ServicePrincipal.Revoke -->
 
 Azure.ServicePrincipal.Revoke 
 
 Azure.ServicePrincipal.Revoke 
 
<!-- Azure.ServicePrincipal.Revoke->Azure.ServicePrincipal -->
 
 Azure.ServicePrincipal.Revoke->Azure.ServicePrincipal 
 
 
 servicePrincipal

In [7]:
var servicePrincipal = await jinagaClient.Fact(new ServicePrincipal(
    new User("-----BEGIN PUBLIC KEY-----\r\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA0mV76/G8fGfJqSaoxfU9\r\nCVyjASlWIn2n\u002BAHnI3jdLsA6Wua6tDTpwuVt4fr9kSZTmxYYJUnBN3MVx4P9K5p9\r\nobGjtT5czi5dzdeKu2vitF5m\u002BiAdtSP1TAIFKNjPQOvC\u002BJDMFTeQaMtLobCipXJZ\r\nl6hbSNflEdqFHFH8cm\u002B2/vLL/6P9nnpWviEnyYq7Pilnb7izttjiR5cAZtN2g0q3\r\nYlDuvQ02mGVEEuJQoMXnDlHbcqxKjoc9ObSdXI7QQgpxcndRjaSAJIrxN9jgSxHU\r\nLegGKiOR2962MT0lJgYJFr\u002B24latHYJ\u002BAt\u002BM2pTW9uuM38Ks3RoVHB4ABbOnhLwX\r\nrQIDAQAB\r\n-----END PUBLIC KEY-----\r\n"),
    environment,
    DateTime.UtcNow
));

servicePrincipal

ServicePrincipal { user = Jinaga.User, environment = Environment { creator = Jinaga.User, environmentId = test }, createdAt = 5/31/2024 4:02:18 PM } user Jinaga.User publicKey -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA0mV76/G8fGfJqSaoxfU9
CVyjASlWIn2n+AHnI3jdLsA6Wua6tDTpwuVt4fr9kSZTmxYYJUnBN3MVx4P9K5p9
obGjtT5czi5dzdeKu2vitF5m+iAdtSP1TAIFKNjPQOvC+JDMFTeQaMtLobCipXJZ
l6hbSNflEdqFHFH8cm+2/vLL/6P9nnpWviEnyYq7Pilnb7izttjiR5cAZtN2g0q3
YlDuvQ02mGVEEuJQoMXnDlHbcqxKjoc9ObSdXI7QQgpxcndRjaSAJIrxN9jgSxHU
LegGKiOR2962MT0lJgYJFr+24latHYJ+At+M2pTW9uuM38Ks3RoVHB4ABbOnhLwX
rQIDAQAB
-----END PUBLIC KEY-----
 environment Environment { creator = Jinaga.User, environmentId = test } creator Jinaga.User publicKey -----BEGIN PUBLIC KEY-----
MIIBCgKCAQEAoCTbrASql8xsTby30NwsizfdDVrfg+8B6y/4brCoyjbGMRpQgh+a
iDvcbNUsafgAyeUgxqOcpXmdyeHL3A4z0KjI1Ti5Us6RADSqve7yt21WZXxpGcmr
YMlxwCJ5vmaxGN4DYAmWhZI7+4zZ6wBj+JarM8kEOz+H3RMsnp05qAoLmx9an+pg
QT8X7pBc24b+uS9sAHl/hboQ/Rd7ivmErbvlyZiXktj8epuETM7VVn4aPMQqezT9
NCQA6kHRC49pTrv9qtGPhI6JXIODG/blMPG6WtNY+7zjEOYAtKf6isq53r1d0hVa
HbdDnPpLQmoqY6/xi17nMRsX3GrVdt7AgQIDAQAB
-----END PUBLIC KEY-----
 environmentId test createdAt 2024-05-31 16:02:18Z

In [9]:
var servicePrincipalsInEnvironment = Given<Environment>.Match((environment, facts) =>
    from servicePrincipal in facts.OfType<ServicePrincipal>()
    where servicePrincipal.environment == environment &&
        !facts.OfType<RevokeServicePrincipal>().Any(revoke => revoke.servicePrincipal == servicePrincipal)
    select servicePrincipal
);

var servicePrincipals = await jinagaClient.Query(servicePrincipalsInEnvironment, environment);

jinagaClient.RenderFacts(servicePrincipals)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- JYIJkjp124uJ2LkcwrustrWWchH8uBcButv/t2zu2YM9YGIAQliFKfZVAb4f1fdJnzjCqfm+uKJ253OLEIbymg== -->
 
 JYIJkjp124uJ2LkcwrustrWWchH8uBcButv/t2zu2YM9YGIAQliFKfZVAb4f1fdJnzjCqfm+uKJ253OLEIbymg== 
 
 Jinaga.User 
 
 publicKey 
 
 -----BEGIN PUBLIC KE... 
 
<!-- J7p3FGxhTGDeqgniTLY/pD8tK5jcSltkPSU1c3RMaDIFNIadVuH28O2J3rGc3m3jw2t1aNQTgOMDYbPJwVlfhQ== -->
 
 J7p3FGxhTGDeqgniTLY/pD8tK5jcSltkPSU1c3RMaDIFNIadVuH28O2J3rGc3m3jw2t1aNQTgOMDYbPJwVlfhQ== 
 
 Jinaga.User 
 
 publicKey 
 
 -----BEGIN PUBLIC KE... 
 
<!-- 5zH07bGoBwtv56nPl/RCiNlnxaV7f19FZq4J1TR6w1pCiUEBtwuViQop063L7t4CNpnIyXZ1tKZp4lfE/5teGg== -->
 
 5zH07bGoBwtv56nPl/RCiNlnxaV7f19FZq4J1TR6w1pCiUEBtwuViQop063L7t4CNpnIyXZ1tKZp4lfE/5teGg== 
 
 Azure.Environment 
 
 environmentId 
 
 test 
 
<!-- 5zH07bGoBwtv56nPl/RCiNlnxaV7f19FZq4J1TR6w1pCiUEBtwuViQop063L7t4CNpnIyXZ1tKZp4lfE/5teGg==->J7p3FGxhTGDeqgniTLY/pD8tK5jcSltkPSU1c3RMaDIFNIadVuH28O2J3rGc3m3jw2t1aNQTgOMDYbPJwVlfhQ== -->
 
 5zH07bGoBwtv56nPl/RCiNlnxaV7f19FZq4J1TR6w1pCiUEBtwuViQop063L7t4CNpnIyXZ1tKZp4lfE/5teGg==->J7p3FGxhTGDeqgniTLY/pD8tK5jcSltkPSU1c3RMaDIFNIadVuH28O2J3rGc3m3jw2t1aNQTgOMDYbPJwVlfhQ== 
 
 
 creator 
 
<!-- YO0QT/MY+ffQjwau0MH81M8jOJVm9A3c8R5xd8zt4jP+UsyS2nLbTsn/hSNpohOUSN/3tfxpXRP4IntWuSjwdQ== -->
 
 YO0QT/MY+ffQjwau0MH81M8jOJVm9A3c8R5xd8zt4jP+UsyS2nLbTsn/hSNpohOUSN/3tfxpXRP4IntWuSjwdQ== 
 
 Azure.ServicePrincipal 
 
 createdAt 
 
 2024-05-31T16:02:18.... 
 
 
<!-- YO0QT/MY+ffQjwau0MH81M8jOJVm9A3c8R5xd8zt4jP+UsyS2nLbTsn/hSNpohOUSN/3tfxpXRP4IntWuSjwdQ==->JYIJkjp124uJ2LkcwrustrWWchH8uBcButv/t2zu2YM9YGIAQliFKfZVAb4f1fdJnzjCqfm+uKJ253OLEIbymg== -->
 
 YO0QT/MY+ffQjwau0MH81M8jOJVm9A3c8R5xd8zt4jP+UsyS2nLbTsn/hSNpohOUSN/3tfxpXRP4IntWuSjwdQ==->JYIJkjp124uJ2LkcwrustrWWchH8uBcButv/t2zu2YM9YGIAQliFKfZVAb4f1fdJnzjCqfm+uKJ253OLEIbymg== 
 
 
 user 
 
<!-- YO0QT/MY+ffQjwau0MH81M8jOJVm9A3c8R5xd8zt4jP+UsyS2nLbTsn/hSNpohOUSN/3tfxpXRP4IntWuSjwdQ==->5zH07bGoBwtv56nPl/RCiNlnxaV7f19FZq4J1TR6w1pCiUEBtwuViQop063L7t4CNpnIyXZ1tKZp4lfE/5teGg== -->
 
 YO0QT/MY+ffQjwau0MH81M8jOJVm9A3c8R5xd8zt4jP+UsyS2nLbTsn/hSNpohOUSN/3tfxpXRP4IntWuSjwdQ==->5zH07bGoBwtv56nPl/RCiNlnxaV7f19FZq4J1TR6w1pCiUEBtwuViQop063L7t4CNpnIyXZ1tKZp4lfE/5teGg== 
 
 
 environment

## Subscriptions

Azure informs the landing page app of subscription activities by and related to a user.
It does so through webhook calls.
The landing page app also permits that user to take actions on the subscription through its own user interface.

In [ ]:
[FactType("Azure.Subscription")]
public record Subscription(Environment environment, Guid subscriptionId);

[FactType("Azure.Subscription.Unsubscribe")]
public record Unsubscribe(Subscription subscription);

Renderer.RenderTypes(typeof(Subscription), typeof(Unsubscribe))

This model also represents the user by their identifier.
The model does not treat Azure users as Jinaga users.
It records information _about_ the user, not _by_ the user.

A user can transfer a subscription to another user.
This is modeled using the Entity Reference pattern, a kind of mutable property.

In [ ]:
[FactType("Azure.UserIdentity")]
public record UserIdentity(Environment environment, string userId);

[FactType("Azure.Subscription.UserIdentity")]
public record SubscriptionUserIdentity(Subscription subscription, UserIdentity userIdentity, SubscriptionUserIdentity[] prior);

Renderer.RenderTypes(typeof(SubscriptionUserIdentity))

## Subscription Activities

A user can adjust the plan to which a subscription applies.
They can also adjust the quantity of the plan.
These are captured as mutable properties.

In [ ]:

[FactType("Azure.Plan")]
public record Plan(Environment environment, string planId);

[FactType("Azure.SubscriptionPlan")]
public record SubscriptionPlan(Subscription subscription, Plan plan, SubscriptionPlan[] prior);

[FactType("Azure.SubscriptionQuantity")]
public record SubscriptionQuantity(Subscription subscription, int quantity, SubscriptionQuantity[] prior);

Renderer.RenderTypes(typeof(Plan), typeof(SubscriptionPlan), typeof(SubscriptionQuantity))

The user can activate and deactivate their subscription.
Azure can also choose to suspend and reinstate the subscription.
Finally, the user periodically renews their subscription.
Renewal occurs within a monthly period.

In [ ]:

[FactType("Azure.Subscription.Activate")]
public record Activate(Subscription subscription, DateTime activatedAt);

[FactType("Azure.Subscription.Deactivate")]
public record Deactivate(Activate activate, DateTime deactivatedAt);

[FactType("Azure.Subscription.Suspend")]
public record Suspend(Subscription subscription, DateTime suspendedAt);

[FactType("Azure.Subscription.Reinstate")]
public record Reinstate(Suspend suspend, DateTime reinstatedAt);

[FactType("Azure.Month")]
public record Month(Environment environment, int year, int month);

[FactType("Azure.Subscription.Renew")]
public record Renew(Subscription subscription, Month month, DateTime renewedAt);

Renderer.RenderTypes(typeof(Suspend), typeof(Reinstate), typeof(Renew), typeof(Activate), typeof(Deactivate))